In [1]:
import pandas as pd
import itertools
from itertools import combinations
from datetime import datetime, date
from data_manager import DataManager
import analyze
from models import Player

dm = DataManager()

class Prop:
    def __init__(self, name, team, stat, threshold, odds, bet_type):
        self.name = name
        self.team = team
        self.stat = stat
        self.n = threshold
        self.odds = odds
        self.bet_type = bet_type
        self.probability = self.get_prop_probability()
        self.ev, self.house_prob = self.get_ev_and_implied_prob()
        self.print_out = f"""
            PLAYER: {self.name}
              STAT: {self.stat}
            THRESH: {self.n}
              ODDS: {self.odds}
              TYPE: {self.bet_type}
              PROB: {self.probability}
                EV: {self.ev}
        HOUSE_PROB: {self.house_prob}
            """
        print(self.print_out)
        self.entry = {
            "PLAYER": self.name,
              "TEAM": self.team,
              "STAT": self.stat,
            "THRESH": self.n,
              "ODDS": self.odds,
              "TYPE": self.bet_type,
              "PROB": self.probability,
                "EV": self.ev,
        "HOUSE_PROB": self.house_prob
        }
    
    def get_prop_probability(self, last_n_games=25):
        player_id = dm.get_player_id(self.name)
        data = dm.get_and_save_player_data(player_id, self.name).sort_values(by='date', ascending=False).head(last_n_games).copy()
        # print(data.head())
        if self.bet_type == "over":
            return analyze.estimate_probability_poisson_over(data, self.stat, self.n)
        elif self.bet_type == "under":
            return analyze.estimate_probability_poisson_under(data, self.stat, self.n)
        else:
            raise ValueError("Invalid bet type. Use 'over' or 'under'.")
        
    def get_ev_and_implied_prob(self):
        odds = self.american_to_decimal(self.odds)
        house_probability = analyze.estimate_implied_probability(odds)
        ev = analyze.calculate_ev(self.probability, odds, 5)
        return ev, house_probability
    
    @staticmethod
    def american_to_decimal(american_odds):
        """Convert American odds to decimal odds."""
        if american_odds > 0:
            return 1 + (american_odds / 100)
        else:
            return 1 + (100 / abs(american_odds))
    

def extract_raw_data(file_path): # .csv
    # gets input from A1
    # Sample input text (use the content of your file here)
    raw_input = pd.read_csv(file_path)
    list_of_raw_input = list(raw_input.iloc[:, 0])
    return list_of_raw_input


def load_available_props():
    
    raw_input = extract_raw_data("prop_lines/prop_lines.csv")
    print("Received raw input.")
    stat_names = {
             'PointsSGP': "points",
            'AssistsSGP': "assists",
        'Threes MadeSGP': "fg3m",
           'ReboundsSGP': "rebounds",
   'Field Goals MadeSGP': "fgm",
             'StealsSGP': "steals",
             'BlocksSGP': "blocks",
        }
    #debug stat_name_inputs = extract_raw_data("prop_lines/player_prop_categories.csv")
    players = dm.query_players()
    player_names = [player.name for player in players]
    teams = dm.query_teams()
    team_names = [team.nickname for team in teams]
    row_of_interest = 0
    current_category = None
    current_player = None
    current_team = None
    records = []
    for _, item in enumerate(raw_input):
        if item is None: 
            continue
        if item in stat_names:
            current_category = stat_names[item]
            print(f"Loading {current_category} props.")
        if item != current_team:
            if item in team_names:
                current_team = item
        if item in player_names:
            current_player = item 
            assert current_team
            record = [current_player, current_team, current_category]
            row_of_interest = 6


        if row_of_interest:
            row_of_interest -= 1
            if row_of_interest < 5:
                record.append(item)
                if row_of_interest == 1:
                    records.append(record)
                    record = []
    df = pd.DataFrame.from_records(records, columns=["player_name", "team", "stat", "over_threshold", "over_odds", "under_threshold", "under_odds"])
    
    df['player_name'] = df['player_name'].astype(str)
    df['team'] = df['team'].astype(str)
    df['stat'] = df['stat'].astype(str)
    df['over_threshold'] = df['over_threshold'].str.extract(r'(\d+\.\d+)').astype(float)
    df['under_threshold'] = df['under_threshold'].str.extract(r'(\d+\.\d+)').astype(float)
    df['over_odds'] = df['over_odds'].astype(int)
    df['under_odds'] = df['under_odds'].astype(int)

    return df


def get_analyzed_props(available_props):
    props = []
    for _, row in available_props.iterrows():
        print(row)
        for bet_type in ["over", "under"]:
            prop = Prop(
                    name=row["player_name"], 
                    team=row["team"],
                    stat=row["stat"], 
                threshold=row[f"{bet_type}_threshold"], 
                    odds=row[f"{bet_type}_odds"], 
                bet_type=bet_type
                )
            props.append(prop)    
            print("Prop object created.")

    return props


def generate_heterogenous_combinations(df, n):

    # Generate all combinations of n rows
    combinations = list(itertools.combinations(df.index, n))

    # Function to evaluate heterogeneity of a combination
    def evaluate_heterogeneity(comb, df):
        comb_list = list(comb)
        players = df.loc[comb_list, 'PLAYER']
        stats = df.loc[comb_list, 'STAT']
        teams = df.loc[comb_list, 'TEAM']
        # Calculate a simple heterogeneity score (you can define your own logic)
        player_score = len(set(players))
        stat_score = len(set(stats))
        team_score = len(set(teams))
        return player_score + stat_score + team_score

    # Evaluate all combinations and sort them by heterogeneity score
    comb_scores = [(comb, evaluate_heterogeneity(comb, df)) for comb in combinations]
    comb_scores_sorted = sorted(comb_scores, key=lambda x: x[1], reverse=True)

    # Select the most heterogeneous combinations (you can define how many you want)
    top_combinations = comb_scores_sorted # Top 5 combinations for example

    # Display the most heterogeneous combinations
    for comb, score in top_combinations:
        print(f"Combination: {comb}, Score: {score}")
        print(df.loc[list(comb)])
        print()

    # Optional: Convert combinations to DataFrame
    top_comb_dfs = [(df.loc[list(comb)], score) for comb, score in top_combinations]

    return top_comb_dfs


def generate_unique_combinations(df, combo_length):
    all_combinations = list(combinations(df.index, combo_length))
    
    unique_combinations = set()
    valid_combinations = []
    
    for combo in all_combinations:
        players_in_combo = df.loc[list(combo), 'Player']
        if players_in_combo.duplicated().sum() == 0:
            sorted_combo = tuple(sorted(df.loc[list(combo)].apply(lambda row: (row['Player'], row['Category'], row['Bet']), axis=1)))
            if sorted_combo not in unique_combinations:
                unique_combinations.add(sorted_combo)
                valid_combinations.append(combo)
    
    return valid_combinations


In [2]:
available_props = load_available_props()
analyzed_props = get_analyzed_props(available_props)
print(analyzed_props)
filter_players = ['Naz Reid', 'Rudy Gobert', "Nickeil Alexander-Walker"]
filtered_df = dm.filter_props(analyzed_props, filter_players)
print(filtered_df.head(5))
# parlays = generate_unique_combinations(filtered_df, 4)


def generate_unique_combinations(df, combo_length):
    all_combinations = list(combinations(df.index, combo_length))
    
    unique_combinations = set()
    valid_combinations = []
    
    for combo in all_combinations:
        players_in_combo = df.loc[list(combo), 'PLAYER']
        if players_in_combo.duplicated().sum() == 0:
            # Create a sorted tuple based on all columns for uniqueness
            sorted_combo = tuple(sorted(df.loc[list(combo)].apply(lambda row: (row['PLAYER'], row['TEAM'], row['STAT'], row['THRESH'], row['ODDS'], row['TYPE']), axis=1)))
            if sorted_combo not in unique_combinations:
                unique_combinations.add(sorted_combo)
                valid_combinations.append(combo)
    
    return valid_combinations

# Generate valid and unique combinations of length 2
valid_combos = generate_unique_combinations(filtered_df, 3)

# Create a list of dataframes for each valid combination
combo_dfs = [filtered_df.loc[list(combo)].reset_index(drop=True) for combo in valid_combos]

# Display each dataframe
for i, combo_df in enumerate(combo_dfs):
    print(f"Combination {i+1}:\n{combo_df}\n")


Received raw input.
Loading points props.
Loading assists props.
Loading fg3m props.
Loading rebounds props.
Loading fgm props.
Loading steals props.
Loading blocks props.
player_name        Obi Toppin
team                   Pacers
stat                   points
over_threshold            7.5
over_odds                -135
under_threshold           7.5
under_odds                100
Name: 0, dtype: object

            PLAYER: Obi Toppin
              STAT: points
            THRESH: 7.5
              ODDS: -135
              TYPE: over
              PROB: 0.8618999066487378
                EV: 2.5017214097204965
        HOUSE_PROB: 0.574468085106383
            
Prop object created.

            PLAYER: Obi Toppin
              STAT: points
            THRESH: 7.5
              ODDS: 100
              TYPE: under
              PROB: 0.13810009335126217
                EV: -3.6189990664873783
        HOUSE_PROB: 0.5
            
Prop object created.
player_name        Aaron Nesmith
team    

In [3]:
candidates = []
for i, df in enumerate(combo_dfs):
    
    today = date.today()
    
    probs = list(df['PROB'])
    odds = [odds for odds in list(df['ODDS'])]
    combined_ev, potential_winnings = analyze.analyze_parlay(probs, odds, 1)
    combined_prob = analyze.calculate_combined_probability(probs)
    df['COMBINED EV'] = combined_ev
    df['COMBINED PROB'] = combined_prob
    df['TO WIN'] = potential_winnings

    parlay_tag = f"{today}_{i}_"
    candidate = [parlay_tag]
    props = []
    for _, row in df.iterrows():
        prop_info = f"{row['PLAYER']}_{row['STAT']}_{row['THRESH']}_{row['ODDS']}_{row['TYPE']}_P{row['PROB']:.3f}_EV{row['EV']:.3f}_HOUSE_PROV{row['HOUSE_PROB']:.3f}"
        props.append(prop_info)
    candidate.append(" + ".join(props))
    candidate += [combined_prob, combined_ev, potential_winnings]
    candidates.append(candidate)


Combined Probability: 0.6181
Combined Odds: 3.48
Expected Value (EV): $1.15
Potential Winnings: $3.48
Combined Probability: 0.6146
Combined Odds: 3.85
Expected Value (EV): $1.36
Potential Winnings: $3.85
Combined Probability: 0.5964
Combined Odds: 3.53
Expected Value (EV): $1.10
Potential Winnings: $3.53
Combined Probability: 0.5750
Combined Odds: 3.48
Expected Value (EV): $1.00
Potential Winnings: $3.48
Combined Probability: 0.5707
Combined Odds: 3.23
Expected Value (EV): $0.85
Potential Winnings: $3.23
Combined Probability: 0.5649
Combined Odds: 3.77
Expected Value (EV): $1.13
Potential Winnings: $3.77
Combined Probability: 0.5479
Combined Odds: 3.77
Expected Value (EV): $1.07
Potential Winnings: $3.77
Combined Probability: 0.5434
Combined Odds: 3.77
Expected Value (EV): $1.05
Potential Winnings: $3.77
Combined Probability: 0.5404
Combined Odds: 2.88
Expected Value (EV): $0.56
Potential Winnings: $2.88
Combined Probability: 0.5303
Combined Odds: 3.85
Expected Value (EV): $1.04
Potent

In [6]:
candidates_df = pd.DataFrame.from_records(candidates, columns = ["parlay_tag", "props", "COMBINED_PROB", "COMBINED_EV", "TO WIN"]).sort_values(by="COMBINED_EV", ascending=False)
candidates_df.to_csv(f"parlays/{datetime.now().strftime('%Y%m%d%H%M%S')}available_parlays.csv")